In [1]:
from collections import namedtuple

In [2]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path



In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories


In [5]:
from pathlib import Path

class Configuration:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifact_root])
    
    def get_data_ingestion_config(self) ->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config




In [6]:
from pathlib import Path
from zipfile import ZipFile
import os
from urllib import request
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=self.config.source_url.split('/')[-1]) as t:
                            request.urlretrieve(self.config.source_url,filename=self.config.local_data_file,
                             reporthook=t.update_to)
            # filename, header = request.urlretrieve(self.config.source_url,filename=self.config.local_data_file)
    
    def _get_updated_list_of_files(list_of_files: list):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath=os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files=zf.namelist()
            updated_list_of_files=self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)




In [7]:
try:
    config=Configuration()
    config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-09-18 21:42:54,833: INFO: common]: yaml file: configs\confIg.yaml loaded successfully
[2022-09-18 21:42:54,836: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-18 21:42:54,839: INFO: common]: created directory at: artifacts
[2022-09-18 21:42:54,840: INFO: common]: created directory at: artifacts/data_ingestion


kagglecatsanddogs_5340.zip:   0%|          | 1.82M/825M [00:26<3:22:08, 67.9kB/s] 


KeyboardInterrupt: 

In [ ]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

: 